In [11]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text as query_text
from sqlalchemy.sql import func
import os

import pandas as pd
import numpy as np
import json, re

app = Flask(__name__)
# Configure SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(app.root_path, '../douban.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.app_context().push()

db = SQLAlchemy(app)

# many to many relation
# https://flask-sqlalchemy.palletsprojects.com/en/2.x/models/
movie_actor_association = db.Table(
    'movie_actor_association',
    db.Column('movie_id', db.Integer, db.ForeignKey('douban_movies.movie_id')),
    db.Column('person_id', db.Integer, db.ForeignKey('persons.person_id')),
)


class DoubanMovie(db.Model):
    __tablename__ = 'douban_movies'

    movie_id = db.Column(db.Integer, primary_key=True)
    movie_name = db.Column(db.String(20))
    release_date = db.Column(db.String(15))
    country = db.Column(db.String(20))
    movie_type = db.Column(db.String(10))
    release_year = db.Column(db.Integer)
    description = db.Column(db.Text)
    douban_url = db.Column(db.String(255))
    poster  = db.Column(db.String(255))
    douban_rate  = db.Column(db.Float)
    rating_count = db.Column(db.Integer)
    
    # actors = db.relationship('Person', secondary=movie_actor_association, backref='movies', lazy='dynamic')
    persons = db.relationship('Person', secondary=movie_actor_association, backref='movies', lazy='dynamic')
 
    def __repr__(self):
        return f'<Douban {self.movie_name}>' 
    
class Person(db.Model):
    __tablename__ = 'persons'

    person_id = db.Column(db.Integer, primary_key=True)
    person_name = db.Column(db.String(20))
    person_name_en = db.Column(db.String(20))
    gender = db.Column(db.String(2))
    birth_date = db.Column(db.String(20))
    birth_place = db.Column(db.String(20))
    birth_year = db.Column(db.Integer())
    
    def __repr__(self):
        return f'<Person {self.person_name}>'    


In [3]:
movies = DoubanMovie.query.all()

In [4]:
persons = Person.query.all()

In [8]:
associations = movie_actor_association

In [2]:
movies = pd.read_sql('douban_movies', db.engine)
persons = pd.read_sql('persons', db.engine)
associations = pd.read_sql('movie_actor_association', db.engine)

AttributeError: 'OptionEngine' object has no attribute 'execute'

In [9]:
movie_actor = {}

for movie_id in associations.movie_id.unique():
    movie_actor[movie_id] = associations.loc[associations.movie_id==movie_id, 'person_id'].values.tolist()[:3]
movie_actor = pd.DataFrame.from_dict(movie_actor, orient='index').stack().droplevel(-1).reset_index()
movie_actor.columns = ['movie_id', 'person_id']

movie_name = movies.set_index('movie_id')['movie_name'].to_dict()
movie_rate = movies.set_index('movie_id')['douban_rate'].to_dict()
rate_count = movies.set_index('movie_id')['rating_count'].to_dict()
person_group = movie_actor.groupby('person_id')['movie_id']
actors = pd.concat([
    person_group.apply(lambda x: [mid for mid in x]),
    person_group.apply(lambda x: [movie_name[mid] for mid in x]),
    person_group.apply(lambda x: [movie_rate[mid] for mid in x]),
    person_group.apply(lambda x: [rate_count[mid] for mid in x]),
    person_group.apply(lambda x: [f"{movie_name[mid]}（{movie_rate[mid]})/n" for mid in x]),
], keys =['movie_id', 'movie_name', 'movie_rate', 'rate_count', 'movies'], 
axis=1).join(persons.set_index('person_id'))
actors['max_rate'] = actors.movie_rate.apply(max)
actors['top_count'] = actors.movie_rate.apply(len)
actors = actors.sort_values(['max_rate', 'top_count'], ascending=False)

AttributeError: 'Table' object has no attribute 'movie_id'

In [ ]:
actors.columns

In [ ]:
cols = ['person_name', 'gender', 'birth_date', 'birth_place',  'birth_year', 'max_rate', 'top_count', 'movies']
actors[cols]

In [10]:
()# https://plotly.com/python/scatter-plots-on-maps/
import plotly.graph_objects as go

with open('../helpers/china.json') as f:
    json_data = json.load(f)['features']
provinces = pd.DataFrame.from_dict(
    data={p['properties']['name']:p['properties']['center'] for p in json_data if p['properties']['name']!=''},
    orient='index', columns=['lon', 'lat']
)
provinces.index = provinces.index.str.replace('市|省|自治区|壮族|回族|维吾尔', '', regex=True)
provinces =  provinces.join(actors.loc[actors.birth_place.str.contains('中国'), 'birth_place'].str.split(',', expand=True)[1].value_counts())
provinces = provinces.loc[provinces['count'].notna()]

provinces['text'] = provinces.apply(lambda x: f"{x.name}: {x['count']:0.0f}", axis=1)

import plotly.express as px
px.set_mapbox_access_token('pk.eyJ1Ijoia2FyaWJ1bnlhaHVhIiwiYSI6ImNsYW0xcGp4dDBhdW8zcG1pcHcxdDR1OGsifQ.I3r8tCO7g08pzM1kFYUwfg')
provinces = provinces.reset_index()
df = px.data.carshare()
fig = px.scatter_mapbox(
    provinces, lat="lat", lon="lon", size="count", hover_name='index', size_max=20, zoom=3)
fig.update_layout(margin = dict(l = 0, r = 0, t = 0, b = 0), coloraxis_showscale=False);
html_string = fig.to_html(full_html=False)
fig.show()

UnicodeDecodeError: 'gbk' codec can't decode byte 0x82 in position 115: illegal multibyte sequence

In [ ]:
html_string